# 1. Import Libary 

In [4]:
import pandas as pd
import pyodbc

# 2. Import Data Frame From SQL SERVER

In [5]:
conn = pyodbc.connect( #CONNECTION
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df_associated_rule = pd.read_sql_query('''
                                        
SELECT *
FROM [vti_dataset].[dbo].[use_case_3_dirty_data]
                                       
    ; ''', conn)
df_associated_rule

C:\Users\Admin-s\AppData\Local\Temp\ipykernel_16892\3665155895.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_associated_rule = pd.read_sql_query('''


,Member_number,Date,itemDescription,Day,Season,Vacation
0,100000000,2015-07-21,tropical fruit,Sun,Winter,True
1,2552,2015-01-05,whole milk,Mon,Summer,False
2,2300,2015-09-19,pip fruit,Tue,Summer,False
3,-1000000,2015-12-12,other vegetables,Wed,Summer,False
4,3037,2015-02-01,whole milk,Thu,Summer,False
...,...,...,...,...,...,...
38779,4471,2014-10-08,sliced cheese,Sat,Winter,True
38780,2022,2014-02-23,candy,Sun,Winter,True
38781,1097,2014-04-16,cake bar,Mon,Winter,True
38782,1510,2014-12-03,fruit/vegetable juice,Tue,Winter,True


# 3. CLEARNING DATA PROCESS

## 3.1 Remove Duplicate Rows

In [8]:
df_associated_rule = df_associated_rule.drop_duplicates()
df_associated_rule

,Member_number,Date,itemDescription,Day,Season,Vacation
0,100000000,2015-07-21,tropical fruit,Sun,Winter,True
1,2552,2015-01-05,whole milk,Mon,Summer,False
2,2300,2015-09-19,pip fruit,Tue,Summer,False
3,-1000000,2015-12-12,other vegetables,Wed,Summer,False
4,3037,2015-02-01,whole milk,Thu,Summer,False
...,...,...,...,...,...,...
38779,4471,2014-10-08,sliced cheese,Sat,Winter,True
38780,2022,2014-02-23,candy,Sun,Winter,True
38781,1097,2014-04-16,cake bar,Mon,Winter,True
38782,1510,2014-12-03,fruit/vegetable juice,Tue,Winter,True


## 3.2 Detect & Remove Outliers Using IQR

In [13]:
# --- Calculate Q1 (25th percentile) and Q3 (75th percentile) ---
Q1 = df_associated_rule['Member_number'].quantile(0.25)
Q3 = df_associated_rule['Member_number'].quantile(0.75)

# --- Calculate IQR ---
IQR = Q3 - Q1
print(f"\nQ1: {Q1}, Q3: {Q3}, IQR: {IQR}")

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_associated_rule = df_associated_rule[(df_associated_rule['Member_number'] >= lower_bound) & (df_associated_rule['Member_number'] <= upper_bound)]


Q1: 2002.0, Q3: 4007.0, IQR: 2005.0


## 3.3 Remove Irrelevant Data Using Biz Sense

In [25]:
df_associated_rule = df_associated_rule[['Member_number','Date','itemDescription']] #OBJECTIVES > 1 => LIST

In [26]:
df_associated_rule

,Member_number,Date,itemDescription
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
4,3037,2015-02-01,whole milk
5,4941,2015-02-14,rolls/buns
6,4501,2015-05-08,other vegetables
...,...,...,...
38779,4471,2014-10-08,sliced cheese
38780,2022,2014-02-23,candy
38781,1097,2014-04-16,cake bar
38782,1510,2014-12-03,fruit/vegetable juice


## 3.4 Standardization Capitalization

In [29]:
df_associated_rule.rename(columns={'Member_number': 'member_number', 'Date': 'date', 'itemDescription': 'item_description'}, inplace=True)
### { } DICTIONARY KEY VALUE

C:\Users\Admin-s\AppData\Local\Temp\ipykernel_16892\3447351722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_associated_rule.rename(columns={'Member_number': 'member_number', 'Date': 'date', 'itemDescription': 'item_description'}, inplace=True)


## 3.5 Covert Data Type

In [33]:
df_associated_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38446 entries, 1 to 38783
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   member_number     38446 non-null  int64 
 1   date              38425 non-null  object
 2   item_description  38425 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [34]:
# --- Convert 'date' column to datetime ---
df_associated_rule['date'] = pd.to_datetime(df_associated_rule['date'])  # errors='coerce' will set invalid parsing to NaT

# --- Convert 'item_description' to string ---
df_associated_rule['item_description'] = df_associated_rule['item_description'].astype(str)


C:\Users\Admin-s\AppData\Local\Temp\ipykernel_16892\1048403363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_associated_rule['date'] = pd.to_datetime(df_associated_rule['date'])  # errors='coerce' will set invalid parsing to NaT
C:\Users\Admin-s\AppData\Local\Temp\ipykernel_16892\1048403363.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_associated_rule['item_description'] = df_associated_rule['item_description'].astype(str)


In [35]:
df_associated_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38446 entries, 1 to 38783
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   member_number     38446 non-null  int64         
 1   date              38425 non-null  datetime64[ns]
 2   item_description  38446 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.2+ MB


## 3.6 Clear Format

## 3.7 Fix Error

## 3.9 Languages Transaltion

## 3.10 Handle Missing Value

In [36]:
df_associated_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38446 entries, 1 to 38783
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   member_number     38446 non-null  int64         
 1   date              38425 non-null  datetime64[ns]
 2   item_description  38446 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.2+ MB


In [40]:
df_associated_rule.dropna(inplace=True)

C:\Users\Admin-s\AppData\Local\Temp\ipykernel_16892\980798460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_associated_rule.dropna(inplace=True)


In [41]:
df_associated_rule

,member_number,date,item_description
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
4,3037,2015-02-01,whole milk
5,4941,2015-02-14,rolls/buns
6,4501,2015-05-08,other vegetables
...,...,...,...
38779,4471,2014-10-08,sliced cheese
38780,2022,2014-02-23,candy
38781,1097,2014-04-16,cake bar
38782,1510,2014-12-03,fruit/vegetable juice


# 4. Save Clear Data Into Database To Visiualization ti

In [42]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "use_case_3_clear_data"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_associated_rule[c].dtype)}" for c in df_associated_rule.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_associated_rule.columns))})", df_associated_rule.values.tolist())
conn.commit()